<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LSTM - Clase 5 - V5.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [2]:
# Ejemplo con LSTM solo obteniento el resultado de los top 10 productos para ver si esta bien

In [49]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [50]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data_full.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [51]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [52]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [53]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


In [55]:
# Obtener los 10 product_id con mayor cantidad de ventas totales
top_product_ids = data.groupby('product_id')['tn'].sum().sort_values(ascending=False).head(3).index.tolist()
top_product_ids

[20001, 20002, 20003]

In [56]:
# Filtrar el DataFrame "data" para incluir solo los "n" product_id más importantes
data_top = data[data['product_id'].isin(top_product_ids)]

In [57]:
# Calcular las ventas totales por product_id y ordenarlas de mayor a menor
product_sales = data.groupby('product_id')['tn'].sum().sort_values(ascending=False)

In [58]:
# Imprimir el listado de los 10 product_id con mayor cantidad de ventas totales
print("Top 3 product_id por ventas totales:")
print(product_sales.head(3))

Top 3 product_id por ventas totales:
product_id
20001    50340.39558
20002    36337.25439
20003    32004.15274
Name: tn, dtype: float64


In [59]:
# Calcular métricas de control
total_sales = data['tn'].sum()
top_10_sales = product_sales.head(3).sum()
market_share = (top_10_sales / total_sales) * 100

print(f"\nVentas totales: {total_sales:.2f}")
print(f"Ventas totales de los 10 product_id más importantes: {top_10_sales:.2f}")
print(f"Participación de mercado de los 10 product_id más importantes: {market_share:.2f}%")
print(f"Ventas promedio por product_id de los 10 más importantes: {product_sales.head(10).mean():.2f}")


Ventas totales: 1122605.22
Ventas totales de los 10 product_id más importantes: 118681.80
Participación de mercado de los 10 product_id más importantes: 10.57%
Ventas promedio por product_id de los 10 más importantes: 26559.14


In [61]:
# Elegir el método de normalización
print("Selecciona el método de normalización:")
print("1. Escalador estándar (StandardScaler)")
print("2. Escalador MinMax (MinMaxScaler)")
print("3. Sin normalización")
choice = int(input("Ingresa el número de la opción (1-3): "))

if choice == 1:
    scaler = StandardScaler()
elif choice == 2:
    scaler = MinMaxScaler()
else:
    scaler = None

Selecciona el método de normalización:
1. Escalador estándar (StandardScaler)
2. Escalador MinMax (MinMaxScaler)
3. Sin normalización


Ingresa el número de la opción (1-3):  2


In [62]:
# Definir y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Inicializar el scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Preparar los datos de entrenamiento
X = data['tn'].values.reshape(-1, 1)  # Reshape a 2D para el scaler
y = data['tn'].values.reshape(-1, 1)

# Escalar los datos
if not hasattr(scaler, 'n_features_in_'):
    scaler.fit(X)
X_normalized = scaler.transform(X)

# Reshape para LSTM
X_lstm = X_normalized.reshape(-1, 1, 1)

# Definir y compilar el modelo LSTM
# model = Sequential()
#model.add(LSTM(50, return_sequences=False, input_shape=(1, 1)))
#model.add(Dense(1))
#model.compile(optimizer='adam', loss='mean_squared_error')
# Definir la capa de entrada
inputs = Input(shape=(1, 1))  # (timesteps, features)

# Definir las capas del modelo
x = LSTM(50, return_sequences=False)(inputs)
outputs = Dense(1)(x)

# Crear el modelo
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error')
# Entrenar el modelo
model.fit(X_lstm, y, epochs=100, batch_size=32, verbose=1)

# Crear una lista con los product_id más importantes
product_ids = top_product_ids

# Crear un DataFrame vacío para las predicciones
predictions = pd.DataFrame(columns=['product_id', 'tn'])

# Hacer predicciones para cada product_id
for product_id in product_ids:
    product_data = data[data['product_id'] == product_id]
    product_data = product_data.sort_values('periodo')
    
    # Obtener el último valor de tn
    last_tn = product_data['tn'].values[-1]
    
    # Normalizar el último valor de tn
    last_tn_reshaped = np.array([last_tn]).reshape(-1, 1)
    last_tn_normalized = scaler.transform(last_tn_reshaped)
    last_tn_lstm = last_tn_normalized.reshape(1, 1, 1)
    
    # Hacer la predicción
    prediction_normalized = model.predict(last_tn_lstm)[0][0]
    
    # Revertir la escala de la predicción
    prediction = scaler.inverse_transform(np.array([prediction_normalized]).reshape(-1, 1))[0][0]
    
    # Agregar la predicción al DataFrame
    new_row = pd.DataFrame({'product_id': [product_id], 'tn': [prediction]})
    predictions = pd.concat([predictions, new_row], ignore_index=True)

# Guardar el DataFrame de predicciones en un archivo CSV
predictions.to_csv('LSTM_top10.v5.0.2.csv', index=False)
print("Predicciones guardadas en 'LSTM_top10.v5.0.2.csv'")

Epoch 1/100


C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


71672/71672 ━━━━━━━━━━━━━━━━━━━━ 129s 2ms/step - loss: 6.0413
Epoch 2/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 123s 2ms/step - loss: 0.3785
Epoch 3/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 118s 2ms/step - loss: 0.2225
Epoch 4/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 130s 2ms/step - loss: 0.1500
Epoch 5/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 272s 4ms/step - loss: 0.0616
Epoch 6/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 237s 3ms/step - loss: 0.0338
Epoch 7/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 208s 3ms/step - loss: 0.0564
Epoch 8/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 204s 3ms/step - loss: 0.0296
Epoch 9/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 163s 2ms/step - loss: 0.0618
Epoch 10/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 179s 3ms/step - loss: 0.0294
Epoch 11/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 169s 2ms/step - loss: 0.0228
Epoch 12/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 167s 2ms/step - loss: 0.0185
Epoch 13/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 171s 2ms/step - loss: 0.0332
Epoch 14/100
71672/71672 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Preparar los datos
X = data['tn'].values.reshape(-1, 1)  # Reshape a 2D para el scaler
y = data['tn'].values.reshape(-1, 1)

# Escalar los datos
if scaler is not None:
    if not hasattr(scaler, 'n_features_in_'):
        scaler.fit(X)
    X_normalized = scaler.transform(X)
else:
    X_normalized = X

# Reshape para LSTM
X_lstm = X_normalized.reshape(-1, 1, 1)

# Entrenar el modelo
model.fit(X_lstm, y, epochs=100, batch_size=32, verbose=1)

In [65]:
# Crear una lista con los product_id más importantes
product_ids = top_product_ids

# Crear un DataFrame vacío para las predicciones
predictions = pd.DataFrame(columns=['product_id', 'tn'])

# Hacer predicciones para cada product_id
for product_id in product_ids:
    product_data = data[data['product_id'] == product_id]
    product_data = product_data.sort_values('periodo')
    
    # Obtener el último valor de tn
    last_tn = product_data['tn'].values[-1]
    
    # Normalizar el último valor de tn (si se seleccionó una opción de normalización)
    if scaler is not None:
        last_tn_normalized = scaler.transform(np.array([last_tn]).reshape(1, 1))[0][0]
        
        # Hacer la predicción para febrero de 2020
        prediction_normalized = model.predict(np.array([last_tn_normalized]).reshape(1, 1, 1))[0][0]
        
        # Revertir la escala de la predicción
        prediction = scaler.inverse_transform(np.array([prediction_normalized]).reshape(1, 1))[0][0]
    else:
        # Hacer la predicción sin normalizar
        prediction = model.predict(np.array([last_tn]).reshape(1, 1, 1))[0][0]
    
    # Agregar la predicción al DataFrame
    new_row = pd.DataFrame({'product_id': [product_id], 'tn': [prediction]})
    predictions = pd.concat([predictions, new_row], ignore_index=True)

# Guardar el DataFrame de predicciones en un archivo CSV
predictions.to_csv('LSTM_top10.v5.0.2.csv', index=False)

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 